In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from hydra import compose, initialize
from omegaconf import OmegaConf

from utils.metric import score

In [3]:
def hide_submission(sub_df):
    hide_sub_df = sub_df.clone().with_columns(
        [pl.lit(0.0).alias(f"ptend_q0002_{i}") for i in range(12, 22)]
    )
    return hide_sub_df

In [4]:
base_sub_path = "gs://kaggle-leap/kami/20240610_ensemble_nelder_mead2_lb078544.parquet"
base_sub_df = pl.read_parquet(base_sub_path)
base_sub_df.head()

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000003,-0.000065,-0.000085,-0.000086,-0.000096,-0.0001,-0.000082,-0.000064,-0.000049,-0.000041,-0.000028,-0.000018,-0.000009,-0.000004,-4.1657e-7,0.000001,0.000006,0.000004,0.000003,-0.000009,0.000009,-0.000003,-0.000012,-0.000016,-0.000015,-0.000013,-0.000016,-0.000018,-0.000023,-0.000031,-0.000037,-0.000032,-0.000023,-0.000018,-0.000017,-0.000015,…,9.3603e-7,0.000002,-4.3578e-7,-0.000002,-0.000001,-6.4969e-7,-8.2227e-7,-0.000003,-7.8813e-7,-5.8951e-7,1.9074e-7,-0.000003,0.000006,0.000001,0.000002,-0.000005,-0.000006,-0.000026,-0.000048,-0.000048,-0.000018,0.000038,0.000043,0.000042,0.000022,0.000012,0.000009,0.000008,4.1996e-7,-0.136167,402.35946,4.6684e-11,4.1327e-10,-0.033577,-0.049489,0.09329,0.212398
"""test_10""",-0.000015,-0.00005,-0.000034,-0.000052,-0.00009,-0.000133,-0.000142,-0.000097,-0.000046,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,-0.000002,-0.000004,-0.000004,-0.000008,-0.000008,-0.000009,-0.000008,-0.000011,-0.000013,-0.000017,-0.00002,-0.000018,-0.00002,-0.000019,-0.000015,-0.000007,-0.000002,0.000003,…,-4.8681e-7,-3.7356e-7,2.0193e-7,4.7321e-7,3.4525e-7,2.1168e-7,1.7050e-7,6.5687e-8,2.0065e-8,2.7567e-9,1.3472e-7,-8.9901e-7,4.5322e-7,-3.2823e-7,0.000001,-4.6198e-9,-5.7225e-7,-0.000002,-0.000002,-0.000002,-0.000002,-0.000001,-1.4282e-7,9.2288e-7,0.000002,0.000001,8.3161e-7,-0.000002,0.000004,-0.027761,345.154827,2.5575e-11,-5.9339e-10,-0.316943,-0.456919,0.0302,0.172592
"""test_100""",-0.000007,-0.00008,-0.000033,-0.000028,-0.000073,-0.000096,-0.000079,-0.000053,-0.00003,-0.000021,-0.000017,-0.000014,-0.000012,-0.00001,-0.000007,-0.000005,-0.000004,-0.000005,0.000004,-0.000009,-0.000004,-0.000007,-0.000006,-0.000006,-0.000004,-0.000003,-0.000006,-0.000006,-0.000007,-0.000011,-0.000015,-0.000019,-0.000022,-0.000022,-0.000024,-0.000024,…,-0.000002,-0.000002,-7.8204e-7,2.2820e-7,8.2554e-7,0.000001,8.8598e-7,8.5306e-7,6.2661e-7,3.6903e-7,1.7289e-8,-4.6920e-7,-1.9907e-7,0.000002,3.3318e-7,-0.000001,-7.2683e-7,-0.000004,-0.00001,-0.00002,-0.000023,0.000009,0.00001,0.000014,0.00001,0.000004,0.000004,0.000004,0.000012,2.065205,331.870707,3.9388e-12,6.2842e-10,0.947471,0.65194,0.288081,0.205645
"""test_1000""",0.000026,0.000007,0.000033,0.000026,-0.000013,-0.000057,-0.000047,-0.000021,-0.000008,-0.000003,-0.000001,-6.6297e-7,-3.7065e-7,0.000002,0.000003,0.000005,0.000006,0.000004,1.3132e-7,7.6130e-7,-4.3587e-7,-0.000002,-0.00001,-0.000019,-0.000017,-0.000014,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.00002,-0.000018,-0.000017,-0.000016,-0.000017,…,2.6728e-8,-4.6045e-8,6.6996e-7,-0.000001,8.5986e-8,0.000001,-0.000001,1.5021e-7,-9.4691e-7,3.3675e-7,-0.000005,0.000007,0.000003,0.000005,0.000007,0.000007,0.000007,0.00001,0.00001,0.000008,0.000006,0.000004,9.9584e-7,-0.000006,-0.000012,-0.000022,-0.000039,-0.000046,0.000025,2.73912,361.004821,-3.

## timestamp使った予測値で置き換える

In [5]:
time_sub_path = "gs://kaggle-leap/kami/experiments/610_sim6/mid/submission.parquet"
time_sub_df = pl.read_parquet(time_sub_path)
time_sub_df.tail()

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_99994""",-0.000004,-0.000042,-0.000038,-0.000057,-0.000068,-0.000092,-0.000083,-0.000059,-0.000041,-0.000035,-0.000027,-0.000018,-0.00001,-0.000005,-0.000002,3.8469e-7,6.0225e-7,-0.000002,8.6690e-8,-0.000002,-0.000003,-0.000005,-0.00001,-0.000011,-0.000011,-0.000012,-0.00002,-0.000026,-0.000028,-0.000027,-0.000026,-0.00003,-0.000033,-0.000037,-0.000039,-0.000036,…,9.5217e-7,0.000001,4.2774e-7,8.4335e-8,-1.6422e-8,-7.7357e-8,-4.3173e-8,2.4093e-9,-1.2662e-8,-5.0907e-8,2.0592e-9,1.6775e-8,-6.4224e-8,-6.2424e-9,-8.1845e-8,-8.9744e-8,-1.4266e-7,-1.0918e-7,2.9620e-9,-1.0200e-7,-3.4877e-7,-6.6667e-7,-0.000004,0.000008,0.000005,-0.000002,4.1089e-7,0.000005,-0.000011,-0.019108,343.39618,-6.1158e-11,-2.1642e-10,-0.475033,0.748098,0.823573,0.397303
"""test_99995""",0.000029,-0.000017,0.000055,0.000048,0.000047,0.000015,-0.000019,-0.000016,-0.000001,0.000002,0.000005,0.000007,0.000006,0.000003,6.5889e-7,-0.000005,-0.000002,-0.000042,0.000025,0.000033,-0.000036,-0.000051,-0.000007,0.000043,0.000063,-0.000024,0.000003,-0.000016,-0.000042,-0.000039,0.000001,0.00001,-0.000011,0.000002,0.000064,0.000149,…,-6.4174e-7,-0.000003,-0.000006,-0.000005,-0.000006,-0.000025,-0.000039,-0.000026,-0.000031,-0.000045,-0.000046,-0.000035,-0.000023,-0.000016,0.000001,0.000021,0.000031,0.000056,0.000057,0.00006,0.000066,0.000064,0.000069,0.000075,0.000077,0.000077,0.000061,0.000038,-0.00005,50.327766,373.786379,-7.9866e-10,2.6252e-8,17.049208,23.653017,18.294265,9.334201
"""test_99996""",0.000055,0.000017,0.000052,0.000074,0.000104,0.000142,0.000153,0.000124,0.000084,0.000068,0.000051,0.000038,0.000027,0.00002,0.000016,0.000013,0.000006,0.000007,4.7977e-7,-0.000012,0.000017,-0.000003,-0.000006,-0.000002,4.3684e-7,-0.000009,-0.000012,-0.00001,-0.000006,-0.000006,0.000005,0.000015,0.00003,0.000031,0.000053,0.000088,…,-9.8909e-8,-5.8967e-8,0.000001,0.000002,0.000005,0.000005,0.000006,0.000007,0.000008,0.000005,0.000003,0.000003,0.000004,0.000008,0.000006,-1.0059e-7,-0.000004,-0.000006,-0.000007,0.000007,-0.000005,-0.000009,-0.000013,-0.000016,-0.000019,-0.000017,-0.000018,-0.000013,-0.000016,336.090357,376.89074,7.2617e-11,1.9914e-8,132.67483,134.040222,90.260488,46.920994
"""test_99997""",0.000001,-0.000018,-0.000052,-0.000038,-0.000066,-0.000068,-0.000054,-0.000043,-0.000028,-0.000023,-0.00002,-0.000017,-0.000015,-0.000012,-0.000009,-0.000007,-0.000006,-0.000004,-0.000007,-0.000005,-0.000008,-0.000005,0.000017,-0.000026,-0.000002,0.000007,-0.000007,-0.000013,-0.000015,-0.000018,-0.000022,-0.000029,-0.000028,-0.000032,-0.000031,-0.000034,…,-3.0850e-7,-9.0931e-7,-9.7670e-7,-5.0733e-7,6.2314e-8,4.9771e-7,4.3595e-7,0.000001,2.5279e-7,1.8774e-7,8.2489e-7,7.5376e-7,-6.1016e-8,-6.8893e-7,0.000001,0.000009,0.000021,0.000017,0.000006,-0.000008,-0.000011,-0.00001,-0.000012,-0.000004,-0.000005,-0.000011,-0.000005,-0.000004,-0.000001,-1.79692,316.14153,6.3202e

In [6]:
class_exp = "701_multi_class/small"
# class_exp = "700_binary_classify/small"

In [7]:
# 閾値ごとの値をチェック

from sklearn.metrics import precision_score, recall_score

val2_predict_df = pl.read_parquet(
    f"gs://kaggle-leap/kami/experiments/{class_exp}/val2_predict.parquet"
)
val2_label_df = pl.read_parquet(
    f"gs://kaggle-leap/kami/experiments/{class_exp}/val2_label.parquet"
)

valid_preds = val2_predict_df.select(["0"]).to_numpy()  # classの番号
valid_y_class = val2_label_df.select(["0"]).to_numpy() == 0

thresholds = [0.5, 0.7, 0.9, 0.99, 0.999, 0.99999, 0.999999]
for threshold in thresholds:
    y_pred = (valid_preds > threshold).astype(np.int64)
    # リコールと精度を計算
    num_cand = np.sum(y_pred)
    precision = precision_score(valid_y_class, y_pred)
    recall = recall_score(valid_y_class, y_pred)
    print(
        f"Threshold: {threshold:.5f}, Num:{num_cand}, Precision: {precision:.5f}, Recall: {recall:.5f}"
    )

Threshold: 0.50000, Num:165832, Precision: 0.98164, Recall: 0.99967
Threshold: 0.70000, Num:164957, Precision: 0.98662, Recall: 0.99944
Threshold: 0.90000, Num:163894, Precision: 0.99243, Recall: 0.99885
Threshold: 0.99000, Num:162458, Precision: 0.99746, Recall: 0.99511
Threshold: 0.99900, Num:159495, Precision: 0.99945, Recall: 0.97892
Threshold: 0.99999, Num:116563, Precision: 1.00000, Recall: 0.71581
Threshold: 1.00000, Num:54501, Precision: 1.00000, Recall: 0.33469


In [8]:
class_pred_df = pl.read_parquet(
    f"gs://kaggle-leap/kami/experiments/{class_exp}/test_predict.parquet"
)
class_preds = class_pred_df.get_column("0").to_numpy().reshape(-1)
class_pred_df.tail()

index,0,1,2,3,4
i64,f32,f32,f32,f32,f32
624995,1.0,2.0277e-7,1.1019e-7,2.1507e-8,1.0022e-10
624996,0.000012,0.987593,0.012306,0.000088,1.8166e-7
624997,0.99999,0.000005,3.0728e-7,0.000004,1.1756e-9
624998,2.9659e-7,0.999991,0.000007,9.7062e-7,8.0808e-9
624999,4.3638e-7,0.000002,0.000559,0.006239,0.9932


In [10]:
# takoi-san check
print(base_sub_df["sample_id"].to_numpy()[class_preds > 0.999])
df = pl.read_parquet("output/exp152_143.parquet")
df.sort("sample_id")

nears,dist,old_time,new_place,sample_id,time_6_18_66_78,old_place
i64,f32,i64,i64,str,i64,i64
258761,0.005093,20190,329,"""test_100008""",0,329
456820,0.01054,35670,244,"""test_100016""",0,244
235737,0.011838,18390,345,"""test_100023""",0,345
234122,0.00494,18270,266,"""test_100027""",0,266
69637,0.028217,5430,133,"""test_100028""",0,133
…,…,…,…,…,…,…
412236,0.018188,32190,204,"""test_99956""",0,204
419900,0.01401,32790,188,"""test_99969""",0,188
392107,0.007062,30630,43,"""test_99976""",0,43


In [15]:
# 閾値以上の行について、カラムごとにR2を計算して一定値以上であれば置き換えを行う
from sklearn.metrics import r2_score

threshold = 0.999999
replace_index = class_preds > threshold
print(f"replace num:{replace_index.sum()} rate:{replace_index.sum()/len(class_preds)}")

replace_sub_df = base_sub_df.clone()
r2_score_vals = []

for col in base_sub_df.columns[1:]:
    time_pred = time_sub_df[col].to_numpy()
    base_pred = base_sub_df[col].to_numpy()
    r2_score_val = r2_score(
        base_pred[replace_index],
        time_pred[replace_index],
    )
    if r2_score_val > 0.2:
        replace_pred = base_sub_df[col].to_numpy().copy()
        replace_pred[replace_index] = time_pred[replace_index]
        replace_sub_df = replace_sub_df.with_columns(
            pl.Series(name=col, values=replace_pred)
        )
        r2_score_vals.append(r2_score_val)
    else:
        print(col, r2_score_val)

replace num:37579 rate:0.0601264
ptend_t_0 -1.0980285257296458
ptend_t_1 -1.0694982052031352
ptend_t_2 -1.2099677293816518
ptend_t_3 -1.1025231909900648
ptend_t_4 -0.9513752402615518
ptend_t_5 -0.840166219085563
ptend_t_6 -0.7824307275571811
ptend_t_7 -0.7738450543616655
ptend_t_8 -0.7901498511873826
ptend_t_9 -0.7876033488899126
ptend_t_10 -0.7891441445146996
ptend_t_11 -0.8281991730847809
ptend_t_12 -0.9040510896012306
ptend_t_13 -1.0112551246729726
ptend_t_14 -1.1976971246930552
ptend_t_15 -1.5649923328538753
ptend_t_16 -2.363472421630556
ptend_t_17 -4.621125880212753
ptend_t_18 -7.246321444042859
ptend_t_19 -5.721791972250844
ptend_t_20 -2.9765549727013725
ptend_t_21 -2.006911144438943
ptend_t_22 -1.7223968333020254
ptend_t_23 -1.5381211706594287
ptend_t_24 -1.5158370400677827
ptend_t_25 -1.4263914012709042
ptend_t_26 -1.3908081235900087
ptend_t_27 -1.4690782053438847
ptend_t_28 -1.6096503549664938
ptend_t_29 -1.8263072611737008
ptend_t_30 -2.0739651948983515
ptend_t_31 -2.32158388

In [57]:
base_pred[replace_index] = 1

ValueError: assignment destination is read-only

In [16]:
time_pred[304:]

array([ 2.11722729e-05, -1.87040935e-05, -6.29029656e-06, ...,
        5.51110531e-05,  1.06978151e-06,  4.36059185e-05])

In [17]:
base_pred[304:]

array([ 3.94424593e-05,  3.97818179e-05,  4.17194799e-05, ...,
        3.05507264e-05, -1.91269003e-05,  3.53775701e-05])

In [20]:
path = "input/sim6_data/test/id384.npz"
a = np.load(path)

In [23]:
a["x"]

array([[2.08036108e+02, 2.27158595e+02, 2.30732909e+02, ...,
        4.90858386e-07, 4.90858386e-07, 4.90858386e-07],
       [2.13779131e+02, 2.22641431e+02, 2.34139642e+02, ...,
        4.90858386e-07, 4.90858386e-07, 4.90858386e-07],
       [2.20849426e+02, 2.22368835e+02, 2.36797136e+02, ...,
        4.90858386e-07, 4.90858386e-07, 4.90858386e-07],
       ...,
       [2.11430493e+02, 2.36875820e+02, 2.53411973e+02, ...,
        4.90858386e-07, 4.90858386e-07, 4.90858386e-07],
       [2.37398034e+02, 2.25109263e+02, 2.36295219e+02, ...,
        4.90858386e-07, 4.90858386e-07, 4.90858386e-07],
       [2.14821313e+02, 2.31346606e+02, 2.38470580e+02, ...,
        4.90858386e-07, 4.90858386e-07, 4.90858386e-07]])

In [24]:
test_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test.csv", n_rows=500)

In [25]:
test_df[384:]

sample_id,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,state_t_9,state_t_10,state_t_11,state_t_12,state_t_13,state_t_14,state_t_15,state_t_16,state_t_17,state_t_18,state_t_19,state_t_20,state_t_21,state_t_22,state_t_23,state_t_24,state_t_25,state_t_26,state_t_27,state_t_28,state_t_29,state_t_30,state_t_31,state_t_32,state_t_33,state_t_34,state_t_35,…,pbuf_N2O_23,pbuf_N2O_24,pbuf_N2O_25,pbuf_N2O_26,pbuf_N2O_27,pbuf_N2O_28,pbuf_N2O_29,pbuf_N2O_30,pbuf_N2O_31,pbuf_N2O_32,pbuf_N2O_33,pbuf_N2O_34,pbuf_N2O_35,pbuf_N2O_36,pbuf_N2O_37,pbuf_N2O_38,pbuf_N2O_39,pbuf_N2O_40,pbuf_N2O_41,pbuf_N2O_42,pbuf_N2O_43,pbuf_N2O_44,pbuf_N2O_45,pbuf_N2O_46,pbuf_N2O_47,pbuf_N2O_48,pbuf_N2O_49,pbuf_N2O_50,pbuf_N2O_51,pbuf_N2O_52,pbuf_N2O_53,pbuf_N2O_54,pbuf_N2O_55,pbuf_N2O_56,pbuf_N2O_57,pbuf_N2O_58,pbuf_N2O_59
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_100455""",208.036108,227.158595,230.732909,232.859734,238.974486,242.371338,238.3371,232.301199,225.518653,222.664938,220.931735,219.55473,218.522116,217.81895,216.910092,216.251177,215.317696,215.025365,212.180458,212.413629,212.314927,212.411587,213.060662,214.098834,215.240623,216.654261,218.504662,221.013468,224.031618,227.345943,230.829695,234.36038,237.889765,241.301154,244.48481,247.469463,…,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7
"""test_100456""",213.779131,222.641431,234.139642,248.09112,259.115334,265.345904,263.341407,255.022751,244.659451,237.155154,231.599541,227.705521,224.966578,223.26269,222.053267,221.173859,220.142456,219.313927,215.369519,214.711592,210.829823,208.767038,208.560237,209.458643,211.015524,212.965759,215.249396,217.730036,220.361509,223.246607,226.559564,230.073359,233.629443,237.156588,240.694411,244.252419,…,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7
"""test_100458""",220.849426,222.368835,236.797136,247.46863,257.170938,265.812413,261.597356,250.263043,239.940483,233.877353,228.405961,222.707502,216.399564,212.72141,207.88912,200.191655,189.471222,186.1124,189.169765,191.528516,198.259119,204.388877,209.790096,215.67299,221.763447,227.443738,232.92412,237.888033,242.479158,246.826067,251.019566,255.024617,258.622324,261.717114,264.686033,267.593346,…,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7
"""test_100459""",210.303844,217.963133,230.294393,240.131893,255.06261,265.784184,264.537377,250.079453,237.615815,232.274324,227.024494,222.443966,218.326228,213.79039,206.145042,198.183557,192.019738,186.633447,189.394467,192.748124,197.786222,203.243175,209.533219,216.201298,222.496667,227.791011,232.613324,237.422549,242.126486,246.770446,251.273411,255.392277,259.238923,262.794763,266.115944,269.019767,…,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9086e-7,4.9